<a href="https://colab.research.google.com/github/pintu1803/DL/blob/main/SpamDetection_1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Spam detection using ML models
####By: Pintu, 181co139
####Date: 30/01/22

mount Googledrive


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Read the dataset

In [2]:
#Pintu 29/01/22
from sklearn import preprocessing
import pandas as pd

dataset=pd.read_csv('/content/drive/MyDrive/Colab datasets/Spam_dataset.csv', header=0, delimiter=",")

#remove the missing data.
dataset=dataset.dropna()
print("Head of the dataset =>\n",dataset.head()) #dataset.tail()

#Category -> Integer
#Ham = 0
#Spam = 1
# le = preprocessing.LabelEncoder()
# le.fit(dataset.Category)
# dataset['Category'] = le.transform(dataset.Category)

spam_cnt=0
for i in range(0, len(dataset['Category'])):
  if (dataset['Category'][i]=='ham'):
    dataset['Category'][i]="0"
  else:
    dataset['Category'][i]="1"
    spam_cnt+=1


print("\nTail of dataset after labelling =>\n",dataset.tail())
print("\nTotal spam texts are =>", spam_cnt," out of ",len(dataset))

Head of the dataset =>
   Category                                            Message
0      ham  Go until jurong point, crazy.. Available only ...
1      ham                      Ok lar... Joking wif u oni...
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...
3      ham  U dun say so early hor... U c already then say...
4      ham  Nah I don't think he goes to usf, he lives aro...

Tail of dataset after labelling =>
      Category                                            Message
5567        1  This is the 2nd time we have tried 2 contact u...
5568        0               Will ü b going to esplanade fr home?
5569        0  Pity, * was in mood for that. So...any other s...
5570        0  The guy did some bitching but I acted like i'd...
5571        0                         Rofl. Its true to its name

Total spam texts are => 747  out of  5572


PreProcessing


In [3]:
#Pintu 29/01/22
#Preprocessing of the dataset step

#PUNCTUATION 
#library that contains punctuation
import string
string.punctuation
#defining the function to remove punctuation
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

#TOLOWER
#converting text into lower format.
def to_lower(text):
    lowered_text=text.lower()
    return lowered_text

#TOKENIZATION
#defining function for tokenization
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
def tokenization(text):
    tokens = word_tokenize(text)
    return tokens

#REMOVE STOPWORDS
#importing nlp library
import nltk
nltk.download('stopwords')
#Stop words present in the library
stopwords = nltk.corpus.stopwords.words('english')
print("\nFew english stopwords are: ",stopwords[0:10])

#defining the function to remove stopwords from tokenized text
def remove_stopwords(text):
    output= [i for i in text if i not in stopwords]
    return output

#STEMMING
#importing the Stemming function from nltk library
from nltk.stem.porter import PorterStemmer
#defining the object for stemming
porter_stemmer = PorterStemmer()
#defining a function for stemming
def stemming(text):
    stem_text = [porter_stemmer.stem(word) for word in text]
    return stem_text

#LEMMATIZATION
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
#defining the object for Lemmatization
wordnet_lemmatizer = WordNetLemmatizer()
#defining the function for lemmatization
def lemmatizer(text):
    lemmed_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemmed_text


##PRE-PROCESSING##
def text_preprocessing(text):
    punctuation_free=remove_punctuation(text)
    lower_text=to_lower(punctuation_free)
    tokenized_text=tokenization(lower_text)
    stopwords_free=remove_stopwords(tokenized_text)
    stemmed_text=stemming(stopwords_free)
    lemmatized_text=lemmatizer(stemmed_text)
    return [punctuation_free,lower_text,tokenized_text,stopwords_free,stemmed_text,lemmatized_text]

#Apply pre-processing function.
for i in range(0,len(dataset['Message'])):
  dataset['Message'][i]=text_preprocessing(dataset['Message'][i])[5]

#Many other steps include: URL removal, HTML tags removal, Rare words removal, Frequent words removal, Spelling checking, and many more.
#But we are not going for further pre-processing cause we need to detect spams and spam contains URL.

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.

Few english stopwords are:  ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


Verify pre-processing

In [4]:
#Pintu 29/01/22
#verify.

text="Let's See; CONNECTION Misunderstanding swimming; ASSure the Authentication: of Working of tExt Pre-processing"
[punctuation_free,lower_text,tokenized_text,stopwords_free,stemmed_text,lemmatized_text]=text_preprocessing(text)
print("Remove punctuation=> ",punctuation_free)
print("Lowered text      => ",lower_text)
print("Tokens separated  => ",tokenized_text)
print("Remove stopwords  => ",stopwords_free)
print("Stemming          => ",stemmed_text)
print("Lemmatizer        => ",lemmatized_text)

print("\nVerify the proper preprocessing:\n",dataset['Message'][0:7])

#success msg.
print("\nAll texts have been preprocessed successfully!!")

Remove punctuation=>  Lets See CONNECTION Misunderstanding swimming ASSure the Authentication of Working of tExt Preprocessing
Lowered text      =>  lets see connection misunderstanding swimming assure the authentication of working of text preprocessing
Tokens separated  =>  ['lets', 'see', 'connection', 'misunderstanding', 'swimming', 'assure', 'the', 'authentication', 'of', 'working', 'of', 'text', 'preprocessing']
Remove stopwords  =>  ['lets', 'see', 'connection', 'misunderstanding', 'swimming', 'assure', 'authentication', 'working', 'text', 'preprocessing']
Stemming          =>  ['let', 'see', 'connect', 'misunderstand', 'swim', 'assur', 'authent', 'work', 'text', 'preprocess']
Lemmatizer        =>  ['let', 'see', 'connect', 'misunderstand', 'swim', 'assur', 'authent', 'work', 'text', 'preprocess']

Verify the proper preprocessing:
 0    [go, jurong, point, crazi, avail, bugi, n, gre...
1                         [ok, lar, joke, wif, u, oni]
2    [free, entri, 2, wkli, comp, win, f

Create Vocabulary

In [30]:
#pintu 30/01/22

from tensorflow.keras.preprocessing.text import Tokenizer                    
from tensorflow.keras.preprocessing.sequence import pad_sequences

x=dataset['Message']
y=dataset['Category']

#create the dict.
tokenizer = Tokenizer(num_words=8000)
tokenizer.fit_on_texts(x)

#number of unique words in dict.
print("Number of unique words in dictionary=",len(tokenizer.word_index))

#replace words with their index in vocab.
x_train = tokenizer.texts_to_sequences(x)


# Adding 1 because of  reserved 0 index
vocab_size = len(tokenizer.word_index) + 1  

#size of random text in training set.
print("Length of random text=")
for i in range(10):
  print("Length of ",i+1," text is => ",len(x_train[i]))

#Maximum length of each text
maxlen = 30

#pad the short text and truncate longer texts.
x_train = pad_sequences(x_train, padding='post', maxlen=maxlen)

Number of unique words in dictionary= 8145
Length of random text=
Length of  1  text is =>  16
Length of  2  text is =>  6
Length of  3  text is =>  23
Length of  4  text is =>  9
Length of  5  text is =>  8
Length of  6  text is =>  19
Length of  7  text is =>  8
Length of  8  text is =>  16
Length of  9  text is =>  18
Length of  10  text is =>  18


Confirm that text is converted into vector

In [31]:
#pintu 30/01/22
print("First tweet=> ",x[0])
print("Text to seq=> ",x_train[1])
print("\nVerification successful!")

First tweet=>  ['go', 'jurong', 'point', 'crazi', 'avail', 'bugi', 'n', 'great', 'world', 'la', 'e', 'buffet', 'cine', 'got', 'amor', 'wat']
Text to seq=>  [  11  249  560  366    1 1596    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]

Verification successful!


SMOTE: Synthetic Minority Oversampling TEchnique

In [32]:
#Pintu 30/01/22

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
#print(imblearn.__version__)
from collections import Counter
import numpy as np

msg,y= x_train, dataset['Category'].to_numpy()
print("Type of message column and class label => ",type(msg),type(y))
count = Counter(y)
print("\nInitial Count of each class is =>",count)

# over = SMOTE(sampling_strategy=0.2) #sampling_strategy argument tells that minority = 10% of majority.
# X, y = over.fit_resample(X, y)
# under = RandomUnderSampler(sampling_strategy=0.5) #sampling_strategy argument tells that majority = 1/0.5 of minority.
# X, y = under.fit_resample(X, y)

#Using Pipeline instead of doing separately.
over = SMOTE(sampling_strategy=0.2)
under = RandomUnderSampler(sampling_strategy=0.5)
steps_in_order = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps_in_order)

# transform the dataset
X, y = pipeline.fit_resample(msg, y)
# summarize the new class distribution
counter = Counter(y)
print("\nNew count after SMOTE of both class =>",counter)
print("\nSize of X and y => ",len(X),len(y))

Type of message column and class label =>  <class 'numpy.ndarray'> <class 'numpy.ndarray'>

Initial Count of each class is => Counter({'0': 4825, '1': 747})

New count after SMOTE of both class => Counter({'0': 1930, '1': 965})

Size of X and y =>  2895 2895


Download zip glove file

In [12]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2022-01-30 06:06:22--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-01-30 06:06:22--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-01-30 06:06:23--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

Unzip glove file

In [13]:
!unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


Create embeddings of vocab.

In [33]:
#Pintu 30/01/22
import numpy as np

#vocab: 'the': 1, mapping of words with integers in seq. 1,2,3..
#embedding: 1->dense vector

def embedding_for_vocab(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  
    # Adding again 1 because of reserved 0 index
    embedding_matrix_vocab = np.zeros((vocab_size, embedding_dim))
  
    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix_vocab[idx] = np.array(vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix_vocab

#matrix for vocab: word_index
embedding_dim = 50
embedding_matrix_vocab = embedding_for_vocab('/content/glove.6B.50d.txt',tokenizer.word_index,embedding_dim)


Verify the vocabulary and embedding matrix.

In [34]:
#Pintu 30/01/22

print("Type of vocabulary => ",type(tokenizer.word_index))
print("\nVocabulary glance => ",)
stop=0

#see the dict.
for word,index in tokenizer.word_index.items():
  stop+=1
  if(stop==10):
    break
  print(index," => ",word)
  
#dense vector.
print("\nDense vector of first word in dict => \n",embedding_matrix_vocab[1])


Type of vocabulary =>  <class 'dict'>

Vocabulary glance => 
1  =>  u
2  =>  call
3  =>  2
4  =>  im
5  =>  go
6  =>  get
7  =>  ur
8  =>  come
9  =>  4

Dense vector of first word in dict => 
 [-0.25676     0.8549      1.10029995  0.95362997  0.36585    -1.30289996
  1.07539999 -0.18460999 -0.67673999  0.37637001 -0.029637    0.51697999
 -0.19248    -0.41863    -0.71144003  0.12564    -0.42965001  0.61456001
  0.41819     0.27605999 -0.48635    -0.32585001  0.67747998  0.15916
  0.35051    -0.29392999 -0.80439001 -0.15939     0.012475   -0.58403999
  2.13529992 -0.1547     -0.57389998  1.45220006  0.6124     -0.68752003
  1.28390002 -0.54631001 -0.35736999  0.57323003  0.35460001 -0.37465
 -0.74628001 -0.074561   -0.48471001  0.067343   -0.039338   -0.22177
  0.099708    0.55553001]


Create embedding for all tweet texts.

In [51]:
#Pintu 30/01/22

import numpy as np

#tweet_text: [23, 0, 34, ..., 35] tweet converted into seq.
#goal: [[dense vector for 23], ..., [dense vecor for 35]]
def embedded_tweet_text(embedding_matrix_vocab, tweet_text):
    matrix_size = len(tweet_text)   
    
    tweet_text_matrix = np.zeros((matrix_size, embedding_dim))
    
    #traverse pad_seq of each tweet text
    for i in range(0,matrix_size):
      index=tweet_text[i]
      if(index==0):
        continue
      else:
        tweet_text_matrix[i]=embedding_matrix_vocab[index]
    return tweet_text_matrix

#define zero matrix.
n=len(X)
x1=np.zeros((n, 30, 50))

#populate matrix.
for i in range(0,n):
  x1[i]=embedded_tweet_text(embedding_matrix_vocab, X[i])

#ML model don't accept input in 3D shape
#so reshape 3D into 2D: 1D array per tweet.
x2=x1.reshape(len(X),-1)
print("Shape of x2 => ",x2.shape)

Shape of x2 =>  (2895, 1500)


Verifying whole embedding.

In [48]:
#x <- preprocessed
#x_train <- padded seq
#X <- smoted
#x1 <- final embedding

print("First tweet as text =>\n",dataset['Message'][1])
print("First tweet: text->seq =>\n",x_train[1])
print("Dense vector for 1st word of above tweet =>\n",embedding_matrix_vocab[22])
print("First tweet -> 2D matrix of dense vectors =>\n",x1[1])

First tweet as text =>
 ['ok', 'lar', 'joke', 'wif', 'u', 'oni']
First tweet: text->seq =>
 [  11  249  560  366    1 1596    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
Dense vector for 1st word of above tweet =>
 [-3.55859995e-01  5.21300018e-01 -6.10700011e-01 -3.01310003e-01
  9.48620021e-01 -3.15389991e-01 -5.98309994e-01  1.21880002e-01
 -3.19430009e-02  5.56949973e-01 -1.06210001e-01  6.33989990e-01
 -4.73399997e-01 -7.58949965e-02  3.82470012e-01  8.15690011e-02
  8.22139978e-01  2.22200006e-01 -8.37639999e-03 -7.66200006e-01
 -5.62529981e-01  6.17590010e-01  2.02920005e-01 -4.85979989e-02
  8.78149986e-01 -1.65489995e+00 -7.74179995e-01  1.54349998e-01
  9.48230028e-01 -3.95200014e-01  3.73020005e+00  8.28549981e-01
 -1.41039997e-01  1.63950007e-02  2.11150005e-01 -3.60849984e-02
 -1.55870005e-01  8.65830004e-01  2.63090014e-01 -7.10150003e-01
 -3.67700011e-02  1.82819995e-03 -1.77039996e-01  2.7031999

Build Models: k-fold cross validation using accuracy as metric.

In [ ]:
#Pintu 30/01/22

#from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from numpy import mean

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

# X and y are being SMOTE

folds = range(3,10)

# define the model to be evaluate

# get a list of models to evaluate
def get_models():
	models = list()
	models.append(LogisticRegression())
	models.append(SGDClassifier())
	models.append(KNeighborsClassifier())
	models.append(DecisionTreeClassifier())
	models.append(SVC())
	models.append(GaussianNB())
	models.append(RandomForestClassifier())
	models.append(GradientBoostingClassifier())
	return models

models = get_models()
# evaluate each k value for a model
def evaluate_model(model):
  for k in folds:
    cross_validation = KFold(n_splits=k, shuffle=True, random_state=1)
    scores = cross_val_score(model, x2, y, scoring='accuracy', cv=cross_validation, n_jobs=-1)
    print("Average of the accuracy for ",k,"folds => ",mean(scores))

# evaluate each model
for m in models:
  model=m
  print("\nSummary of model:",model)
  evaluate_model(model)


Summary of model: LogisticRegression()
Average of the accuracy for  3 folds =>  0.8818652849740932
Average of the accuracy for  4 folds =>  0.8853132665459297
Average of the accuracy for  5 folds =>  0.8870466321243523
Average of the accuracy for  6 folds =>  0.8790974745782613
Average of the accuracy for  7 folds =>  0.8859971893449102
Average of the accuracy for  8 folds =>  0.8853198986853583
Average of the accuracy for  9 folds =>  0.8853060118805751

Summary of model: SGDClassifier()
Average of the accuracy for  3 folds =>  0.865630397236615
Average of the accuracy for  4 folds =>  0.8670030871980622
Average of the accuracy for  5 folds =>  0.8735751295336789
Average of the accuracy for  6 folds =>  0.8721911520035279
Average of the accuracy for  7 folds =>  0.8721803631794157
Average of the accuracy for  8 folds =>  0.8718463522137709
Average of the accuracy for  9 folds =>  0.8787379415172994

Summary of model: KNeighborsClassifier()
Average of the accuracy for  3 folds =>  0.7

In [54]:
print("Program ended!")

Program ended!
